In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11444,2024-10-28,Kosovo Superliga,15:00,Sigal Prishtina,Vellaznimi,1.21,3.74,157.5,1.83,1.83,-10.5,1.98,1.70,MyEzbUGm,0.826446,0.267380,0.546448,0.546448,0.093826,172.514,68.766924,0.398616,0.6,1.341641,2.236068,164.80,2.150,0.880142,0.409368,-22.0,232.326,116.031958,0.499436,1.2,1.643168,1.369306,164.88,2.980,1.037135,0.348032,-27.0,80,72,2.06,2.29,0.000,0.000,0.722820,0.000000,0.107603,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11445,2024-10-28,Israel Superliga,15:50,Galil Elyon,H. Afula,1.61,2.25,162.5,1.80,1.90,-7.5,2.05,1.60,IHdQVgKa,0.621118,0.444444,0.555556,0.526316,0.065562,180.650,26.174813,0.144892,1.2,1.643168,1.369306,196.98,2.300,0.580603,0.252436,22.0,166.588,21.290212,0.127802,0.0,0.000000,NaN,169.36,2.058,0.281993,0.137023,-36.0,67,73,2.94,2.32,212.218,166.588,0.234481,0.038222,0.174355,-3.23,-0.646,-0.944272,0.388051,0.1,-0.288051,-5.00,-1.000,-1.250000,0.450543,0.1,-0.350543
11446,2024-10-28,Israel Superliga,13:50,Hapoel Beer Sheva,Hapoel Tel-Aviv,7.33,1.08,162.5,1.83,1.83,10.5,1.95,1.72,KtgsSBJO,0.136426,0.925926,0.546448,0.546448,0.062352,166.230,58.792913,0.353684,2.4,1.341641,0.559017,123.24,1.932,0.789189,0.408483,17.0,203.598,78.083053,0.383516,1.8,1.643168,0.912871,108.64,2.534,1.159474,0.457567,33.0,78,97,1.58,1.12,148.936,177.390,1.050991,0.000000,0.088629,3.20,0.640,9.890625,0.550373,0.6,0.049627,-0.92,-0.184,-0.434783,0.621354,0.5,-0.121354
11447,2024-10-28,Grécia Liga De Basquete,15:15,Peristeri,Lavrio,1.27,3.70,159.5,1.87,1.79,-9.5,2.00,1.72,dlBSYLb5,0.787402,0.270270,0.534759,0.558659,0.057672,237.848,171.214748,0.719849,1.8,1.643168,0.912871,185.65,3.218,2.446277,0.760185,-4.0,378.054,392.312365,1.037715,0.6,1.341641,2.236068,127.28,4.846,5.282067,1.089985,-30.0,79,74,2.35,1.72,141.522,379.718,0.691457,0.030912,0.106446,-1.07,-0.214,-1.261682,0.687932,0.6,-0.087932,-3.35,-0.670,-4.029851,0.356703,0.2,-0.156703
11448,2024-10-28,Irã Superliga,11:30,Kalleh,Petro Novin Mahshahr,1.01,9.70,145.5,1.84,1.82,-19.5,1.93,1.74,plxlNjWg,0.990099,0.103093,0.543478,0.549451,0.093192,164.224,26.279964,0.160025,1.2,1.643168,1.369306,126.38,2.058,0.679389,0.330121,-17.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,89,73,1.42,2.56,139.316,0.000,1.147480,0.007728,0.073215,-1.09,-0.218,-0.045872,0.689032,0.6,-0.089032,0.00,0.000,inf,0.000000,0.0,0.000000
11449,2024-10-28,Irã Superliga,09:30,Tabiat,Zob Ahan,1.38,2.72,150.5,1.88,1.78,-7.5,2.00,1.69,n7N6e0EO,0.724638,0.367647,0.531915,0.561798,0.092285,127.900,43.608050,0.340954,2.4,1.341641,0.559017,77.88,1.672,0.454280,0.271698,15.0,166.880,44.725518,0.268010,1.2,1.643168,1.369306,225.76,2.124,0.468113,0.220392,-4.0,66,83,1.18,2.72,132.728,176.038,0.462206,0.038640,0.118809,2.87,0.574,0.662021,0.723677,0.9,0.176323,-1.30,-0.260,-6.615385,0.525579,0.5,-0.025579
11450,2024-10-28,Europa Liga Da Aba 2,16:00,Ilirija,Zlatibor,2.06,1.64,158.5,1.84,1.82,1.5,1.95,1.72,Gvi78mSE,0.485437,0.609756,0.543478,0.549451,0.095193,170.656,65.634430,0.384601,1.8,1.643168,0.912871,264.00,2.034,0.868407,0.426945,42.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,75,90,3.52,1.38,0.000,0.000,0.160532,0.007728,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11452,13:40,Israel Superliga,H. Haifa,Ironi Kiryat Ata,1.60,Back Home
11458,20:15,Uruguai Liga Uruguaia,Cordon,Urupan,1.68,Back Home
11459,20:15,Uruguai Liga Uruguaia,Penarol,Aguada,1.59,Back Home
